In [33]:
import pandas as pd 
import numpy as np
df1 = pd.read_csv("data/results.csv")
df2 = pd.read_csv("data/goalscorers.csv")
df2 =df2[df2["own_goal"]==False]
df3 = pd.read_csv("data/shootouts.csv")
df4 = pd.read_csv("data/former_names.csv")

In [34]:
df1["goal_difference"]=df1["home_score"]-df1["away_score"]
conditions = [
    df1["goal_difference"] > 0,   
    df1["goal_difference"] < 0    
]
choices = [
    df1["home_team"],            
    df1["away_team"]             
]
df1["winner"] = np.select(conditions, choices, default="Draw")


# OVERVIEW

get an overview of the results file

In [3]:
print(df1.head())

         date home_team away_team  home_score  away_score tournament     city  \
0  1872-11-30  Scotland   England           0           0   Friendly  Glasgow   
1  1873-03-08   England  Scotland           4           2   Friendly   London   
2  1874-03-07  Scotland   England           2           1   Friendly  Glasgow   
3  1875-03-06   England  Scotland           2           2   Friendly   London   
4  1876-03-04  Scotland   England           3           0   Friendly  Glasgow   

    country  neutral  
0  Scotland    False  
1   England    False  
2  Scotland    False  
3   England    False  
4  Scotland    False  


get an overview of scorers

In [4]:
print(df2.head())


         date  home_team away_team       team            scorer  minute  \
0  1916-07-02      Chile   Uruguay    Uruguay   José Piendibene    44.0   
1  1916-07-02      Chile   Uruguay    Uruguay  Isabelino Gradín    55.0   
2  1916-07-02      Chile   Uruguay    Uruguay  Isabelino Gradín    70.0   
3  1916-07-02      Chile   Uruguay    Uruguay   José Piendibene    75.0   
4  1916-07-06  Argentina     Chile  Argentina     Alberto Ohaco     2.0   

   own_goal  penalty  
0     False    False  
1     False    False  
2     False    False  
3     False    False  
4     False    False  


get an overview of shooutouts

In [5]:
print(df3.head())

         date    home_team         away_team       winner first_shooter
0  1967-08-22        India            Taiwan       Taiwan           NaN
1  1971-11-14  South Korea  Vietnam Republic  South Korea           NaN
2  1972-05-07  South Korea              Iraq         Iraq           NaN
3  1972-05-17     Thailand       South Korea  South Korea           NaN
4  1972-05-19     Thailand          Cambodia     Thailand           NaN


get an overview of former names

In [6]:
print(df4.head())

          current                former  start_date    end_date
0           Benin               Dahomey  1959-11-08  1975-11-30
1    Burkina Faso           Upper Volta  1960-04-14  1984-08-04
2         Curaçao  Netherlands Antilles  1957-03-03  2010-10-10
3  Czechoslovakia               Bohemia  1903-04-05  1919-01-01
4  Czechoslovakia   Bohemia and Moravia  1939-01-01  1945-05-01


# RESULTS

average_goals home/away

In [7]:
average_home_score=df1["home_score"].mean()
print(average_home_score)
average_away_score=df1["away_score"].mean()
print(average_away_score)

1.7578739753045836
1.181743471739979


average difference

In [8]:
df1["goal_difference"]=df1["home_score"]-df1["away_score"]
average_difference=df1["goal_difference"].mean()
print(average_difference)

0.5761305035646046


### checking rates home/away/draw

home winrate

In [9]:
home_wins=df1[df1["goal_difference"]>0]
home_winrate=len(home_wins)/len(df1)
print(home_winrate)

0.4903950855710558


away winrate

In [10]:
away_wins=df1[df1["goal_difference"]<0]
away_winrate=len(away_wins)/len(df1)
print(away_winrate)

0.2823536663037002


draw rate

In [11]:
draws=df1[df1["goal_difference"]==0]
draw_rate=len(draws)/len(df1)
print(draw_rate)

0.22725124812524397


### checking teams sorted by goals scored

teams sorted by home goals scored

In [12]:
def scored_at_home(dataframe):
    teams_by_goals_scored_at_home=dataframe.groupby("home_team")["home_score"].sum()
    teams_by_goals_scored_at_home = teams_by_goals_scored_at_home.sort_values(ascending=False)
    print(teams_by_goals_scored_at_home.head(10))

scored_at_home(df1)


home_team
Brazil         1495
Germany        1372
Argentina      1337
England        1247
Sweden         1236
Mexico         1197
Netherlands    1129
Hungary        1127
France         1111
South Korea    1106
Name: home_score, dtype: int64


teams sorted by home goals scored against xxx

In [13]:
def scored_at_home_against(dataframe,x):
    df_at_home_against_x=dataframe[dataframe["away_team"]==x]
    scored_at_home(df_at_home_against_x)

scored_at_home_against(df1,"Sweden")

home_team
Denmark        103
Norway          79
Finland         61
Hungary         49
Germany         44
Switzerland     31
Austria         30
Netherlands     26
Italy           21
Spain           21
Name: home_score, dtype: int64


teams sorted by total goals scored

In [14]:
def scored_overall(dataframe):
    teams_by_goals_scored_at_home=dataframe.groupby("home_team")["home_score"].sum()
    teams_by_goals_scored_away=dataframe.groupby("away_team")["away_score"].sum()
    teams_by_total_goals=teams_by_goals_scored_at_home.add(teams_by_goals_scored_away, fill_value=0)
    teams_by_total_goals = teams_by_total_goals.sort_values(ascending=False)
    print(teams_by_total_goals.head(10))

scored_overall(df1)

England        2372.0
Germany        2300.0
Brazil         2290.0
Sweden         2160.0
Argentina      2007.0
Hungary        2002.0
Netherlands    1831.0
South Korea    1783.0
Mexico         1752.0
France         1699.0
dtype: float64


# GOALSCORERS

possible questions:
- most goals scored Y
- most goals for Germany Y
- most last ten minutes goals Y
- most decider goals Y
- most own goals
- most goals against x
- most penalty to goal ratio
- most goals scored in the same minute by one player
- in which minute are the most goals scored
- team with most 100+ Scorers
- most hattricks
- most "useless" goals (already 2+ goals up)
- most goals in the year ...
- most goals in the month ...
- most goals at dates like 01.01., 02.02., ...

## most goals scored

total goals

In [31]:
def goals_scored_players(dataframe):
    goals_per_player=dataframe.groupby("scorer")["scorer"].count()
    goals_per_player=goals_per_player.sort_values(ascending=False)
    print(goals_per_player.head(10))

goals_scored_players(df2)

scorer
Cristiano Ronaldo      108
Robert Lewandowski      63
Romelu Lukaku           60
Harry Kane              58
Lionel Messi            55
Edin Džeko              50
Ali Daei                49
Miroslav Klose          48
Aleksandar Mitrović     47
Carlos Ruiz             47
Name: scorer, dtype: int64


just noticed that there are just results from tournaments and there are no friendlies at all so therefore it could be better to analyse relative values instead of absoulte values

total goals for x

In [19]:
def goals_scored_players_for_x(dataframe,x):
    x_dataframe=dataframe.loc[dataframe['team']==x]
    goals_scored_players(x_dataframe)

goals_scored_players_for_x(df2,"Germany")

scorer
Miroslav Klose           48
Gerd Müller              39
Thomas Müller            35
Karl-Heinz Rummenigge    32
Jürgen Klinsmann         30
Lukas Podolski           29
Michael Ballack          27
Rudi Völler              26
Timo Werner              20
Oliver Bierhoff          19
Name: scorer, dtype: int64


most goals in last x min 

In [26]:
def goals_scored_players_in_min_x(dataframe,minutes,first_or_last: bool): ##True is before that minute and False is after that
    if first_or_last==True:
        min_dataframe=dataframe.loc[dataframe["minute"]<=minutes]
    else:
        min_dataframe=dataframe.loc[dataframe["minute"]>=minutes]

    goals_scored_players(min_dataframe)

goals_scored_players_in_min_x(df2,3,True)

scorer
Damian Mori           4
Milan Galić           3
Clint Dempsey         3
Toni Polster          3
Cristiano Ronaldo     3
Gabriel Batistuta     3
Aleksandr Iashvili    3
Oliver Bierhoff       3
Haykel Guemamdia      3
Oreste Corbatta       3
Name: scorer, dtype: int64


decider goals (first analysis using 2 tables)

In [36]:
#definition decider goal: the goal difference is exactly 1 and last goalscorer is 
#a player of the winning team

#idea: search for all the last goalscorers in the goalscorer db and then checking the score of the game in the game db
#reasons: not all games in game db are included in goalscorer db and because of the elimination of own goals the scores
#         are not accessible in the goalscorer db


# df_last_scorer=df2.group_by(["date", "home_team","away_team"]).last()
# df_last_scorer=df_last_scorer.reset_index()
# df_deciders=df_last_scorer.loc[
#     df1.loc[
#         ...
#     ]
# ]

#found the concept of merge
df_last_scorer=df2.groupby(["date", "home_team","away_team"]).last()
df_decider_matches=df1.loc[abs(df1["goal_difference"])==1]
df_last_scorer=df_last_scorer.reset_index()
df_deciders_merged=pd.merge(
    df_decider_matches,
    df_last_scorer,
    on=["date","home_team","away_team"],
    how="inner"
)
df_deciders_merged=df_deciders_merged.loc[
    df_deciders_merged["team"]==df_deciders_merged["winner"]
]


goals_scored_players(df_deciders_merged)


scorer
Cristiano Ronaldo    13
Gareth Bale           8
David Villa           7
Gabriel Batistuta     7
Gerd Müller           7
Gerson Rodrigues      6
Kylian Mbappé         6
André Ayew            6
Edinson Cavani        6
Clint Dempsey         6
Name: scorer, dtype: int64
